# Meesho Sales Analysis
Synthetic dataset EDA and visuals.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

df = pd.read_csv('meesho_sales.csv', parse_dates=['OrderDate','ShipDate'])
df.head()

## KPIs

In [ ]:
kpis = {
  'orders': len(df),
  'units': int(df['Units'].sum()),
  'net_sales': float(df['NetSales'].sum()),
  'profit': float(df['Profit'].sum()),
}
aov = df['NetSales'].mean()
profit_margin = 100*kpis['profit']/kpis['net_sales'] if kpis['net_sales'] else 0
kpis, aov, profit_margin

## Monthly Trends

In [ ]:
monthly = df.groupby(df['OrderDate'].values.astype('datetime64[M]')).agg(
    Orders=('OrderID','count'), Units=('Units','sum'),
    NetSales=('NetSales','sum'), Profit=('Profit','sum')
).reset_index().rename(columns={'OrderDate':'Month'})
plt.figure(); plt.plot(monthly['Month'].dt.strftime('%Y-%m'), monthly['NetSales']);
plt.xticks(rotation=45, ha='right'); plt.title('Monthly Net Sales'); plt.xlabel('Month'); plt.ylabel('Net Sales (₹)'); plt.show()
plt.figure(); plt.plot(monthly['Month'].dt.strftime('%Y-%m'), monthly['Profit']);
plt.xticks(rotation=45, ha='right'); plt.title('Monthly Profit'); plt.xlabel('Month'); plt.ylabel('Profit (₹)'); plt.show()

## Category & Geography

In [ ]:
cat_sum = df.groupby('Category')['NetSales'].sum().sort_values(ascending=False)
plt.figure(); plt.bar(cat_sum.index, cat_sum.values);
plt.title('Net Sales by Category'); plt.xlabel('Category'); plt.ylabel('Net Sales (₹)'); plt.show()

state_sum = df.groupby('State')['NetSales'].sum().sort_values(ascending=False).head(10)
plt.figure(); plt.bar(state_sum.index, state_sum.values);
plt.title('Top States by Net Sales'); plt.xlabel('State'); plt.ylabel('Net Sales (₹)'); plt.xticks(rotation=30, ha='right'); plt.show()

## Payment Mix & Returns

In [ ]:
pm = df['PaymentMethod'].value_counts()
plt.figure(); plt.bar(pm.index.astype(str), pm.values);
plt.title('Orders by Payment Method'); plt.xlabel('Payment Method'); plt.ylabel('Orders'); plt.show()

ret_rate = df.groupby('Category')['Returned'].mean().sort_values(ascending=False)
plt.figure(); plt.bar(ret_rate.index, ret_rate.values*100);
plt.title('Return Rate by Category (%)'); plt.xlabel('Category'); plt.ylabel('Return Rate (%)'); plt.show()

## Discount vs Profit

In [ ]:
sample = df.sample(min(len(df), 1000), random_state=42)
plt.figure(); plt.scatter(sample['DiscountPct']*100, sample['Profit']);
plt.title('Discount % vs Profit per Order'); plt.xlabel('Discount (%)'); plt.ylabel('Profit (₹)'); plt.show()